In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Introduction to Variational Inference
Consider the problem of learning the conditional distribution $p(\theta|x)$. From Bayes' rule we have
$$p(\theta|x) \propto \frac{p(x, \theta)}{p(x)} = \frac{p(x|\theta)p(\theta)}{\int p(x|\theta)p(\theta) d\theta }$$
This can be an arbitrary conditional distribution, but the canonical motivation comes from Bayesian inference, in which case $p(\theta|x)$ is the posterior distribution of the parameters $\theta$ given data $x$. For notational convenience, I will write $\pi(\theta) := p(\theta|x)$. By "learning" the conditional distribution, we typically mean calculating quantities of interest that summarize the distribution, such as the mean, variance, tail probabilities, etc. All of these quantities of interest represent integrals with respect to the distribution $\pi(\theta)$. In certain simple cases, $\pi(\theta)$ takes the form of a known distribution, for which the quantities of interest are already known. More often, we face the challenge of $\pi(\theta)$ being an unknown, complicated distribution, where the normalizing constant (i.e. the evidence)
$$p(x) = \int p(x|\theta)p(\theta) d\theta$$
is an intractable integral. The challenge is thus to be able to calculate, or approximate, integrals with respect to $\pi(\theta)$ where we only have access to pointwise evaluations of $p(x|\theta)p(\theta)$, which is proportional to the density $\pi(\theta)$. A common solution to this problem is to leverage our knowledge of the density up to a proportionality constant to draw samples distributed according to $\pi(\theta)$ and then approximate integrals with respect $\pi(\theta)$ using sample means. Markov Chain Monte Carlo (MCMC) algorithms represent the state-of-the-art sampling scheme in many applications. However, in certain large-scale settings, the runtime of MCMC can be prohibitively expensive. Variational inference (VI) techniques offer another approach, which can scale to larger settings, at the potential cost of accuracy. 

# The Basic Idea
The basic idea is to re-cast the idea of learning $\pi(\theta)$ as an optimization problem. In particular, we consider a family of surrogate distributions $\{q\}_{q \in Q}$, which are hopefully rich enough to provide a good approximation of $\pi(\theta)$, but simple enough to be something we can understand and easily work with. The idea is to try to find the $q \in Q$ that best approximates $\pi$ in some metric $d(\cdot)$. 

$$q^* := \text{argmin}_{q \in Q} d(q, \pi)$$

Typically, the family $Q$ will be parameterized by some finite set of parameters, but we will defer discussion on this until later. For now, we consider the metric that measures the distance between $\pi$ and its approximation. A reasonable choice might be to consider the KL-divergence

$$KL(\pi||q) = \int \pi(\theta)\log \frac{\pi(\theta)}{q(\theta)} d\theta$$

Since we're interested in approximating the true distribution $\pi$, it appears to make sense to weight the pointwise errors

$$\log \frac{\pi(\theta)}{q(\theta)} = \log \pi(\theta) - \log q(\theta)$$

by the true weights $\pi(\theta)$. However, notice that $KL(\pi||q)$ is an expectation with respect to $\pi$, which is intractable as this is the very problem we're trying to solve. So instead we might consider the reverse KL-divergence,

\begin{align*}
KL(q||\pi) &= \int q(\theta)\log \frac{q(\theta)}{\pi(\theta)} d\theta \\
           &= \int q(\theta)\log \frac{q(\theta)p(x)}{p(x, \theta)} d\theta
\end{align*}

Although $KL(p||\pi)$ is an integral with respect to $q$ instead of $\pi$, we see in the second line above that the reverse KL still depends on the evidence $p(x)$, which is again an intractable integral with respect to $\pi$. However, recalling that the goal is to solve the optimization problem
$$\text{argmin}_{q \in Q} KL(q||\pi)$$
then minimizing the KL-divergence is equivalent to minimizing a function that is equal to the KL-divergence up to an additive constant. It turns out, as is shown below, that the evidence $p(x)$ can be pulled out as such an additive constant.

\begin{align*}
KL(q||\pi) &= \int q(\theta)\log \frac{q(\theta)p(x)}{p(x, \theta)} d\theta \\
           &= \int q(\theta)\left[\log\frac{q(\theta)}{p(x, \theta)} + \log p(x)\right] d\theta \\
           &= \mathbb{E}_q\left[\log\frac{q(\theta)}{p(x, \theta)}\right] + \mathbb{E}_q\left[\log p(x)\right] \\
           &= \mathbb{E}_q\left[\log\frac{q(\theta)}{p(x, \theta)}\right] + \log p(x)
\end{align*}

Note that the evidence $p(x)$ is not a function of $\theta$ and hence is indeed a constant with respect to distributions over $\theta$. Thus, minimizing $KL(q||\pi)$ is equivalent to minimizing $\mathbb{E}_q\left[\log\frac{q(\theta)}{p(x, \theta)}\right]$, or alternatively to maximizing 

$$-\mathbb{E}_q\left[\log\frac{q(\theta)}{p(x, \theta)}\right] = \mathbb{E}_q\left[\log\frac{p(x, \theta)}{q(\theta)}\right]$$

Since this quantity will be the primary objective function in the optimization, we give it a name - the Evidence Lower Bound (ELBO),

$$ELBO(q) := \mathbb{E}_q\left[\log\frac{p(x, \theta)}{q(\theta)}\right]$$
which satisfies

$$\text{argmin}_{q \in Q} KL(q||\pi) = \text{argmin}_{q \in Q} ELBO(q)$$

Why is this function called the Evidence Lower Bound? The above derivations showed that 

$$ KL(q||\pi) = \log p(x) - ELBO(q) $$

Combining this with the fact that $KL(q||\pi) \geq 0$, we arive at the conclusion

$$ ELBO(q) \leq \log p(x) $$

so indeed, $ELBO(q)$ is less than or equal to the (log) evidence, for any $q$. Since we're tryinig to maximize $ELBO(q)$, we know that $\log p(x)$ is the best we can possibly do. 